<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/main_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext Cython
import numpy as np
from math import sin,tan,cos

Make It Work Make It Right Make It Fast

In [ ]:
%%cython

def c_fib_typed(int n):
  """Return the Fibonacci series up to n."""
  cdef int a, b, len_r
  cdef int r[1000]
  a = 0
  b = 1
  len_r = 0
  while b < n:
    r[len_r] = b
    len_r += 1
    a, b = b, a + b
        
  r_as_list  = [item for item in r[:len_r]]
  return r_as_list

In [ ]:
%timeit x = c_fib_typed(2000)

The slowest run took 10.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 285 ns per loop


# Cython and Numpy Arrays

UITLEG MEMORY VIEWS


In [ ]:
%%cython
import numpy as np

arr = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(arr)

cdef int [:, :] arr_view = arr
print(np.asarray(arr_view))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [ ]:
%%cython 
import numpy as np

cdef int [:, :] arr_view = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(np.asarray(arr_view))
print(np.asarray(arr_view[1,1]))
arr_view[1,1] = 10
print(np.asarray(arr_view[1,1]))
arr_view[:,:] = 5
print(np.asarray(arr_view))


[[0 1 2]
 [3 4 5]
 [6 7 8]]
4
10
[[5 5 5]
 [5 5 5]
 [5 5 5]]


## Looping over arrays

UITLEG FOR LOOPS IN C

In [ ]:
x = np.arange(100, dtype=np.dtype("i")).reshape((10, 10))
y = np.full((10,10), 2, dtype=np.dtype("i"))

In [ ]:
def array_op(x,y):
  result = np.zeros(x.shape)
  for i in range(x.shape[0]):
      for j in range(x.shape[1]):
          result[i,j] = x[i,j] * y[i,j] + i
  return result
    

In [ ]:
%timeit result = array_op(x,y)

1000 loops, best of 5: 257 µs per loop


In [ ]:
%%cython 
import numpy as np

def cdef_array_op(x, y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [ ]:
%timeit cdef_array_op(x,y)

1000 loops, best of 5: 274 µs per loop


In [ ]:
%%cython 
import numpy as np

def typed_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [ ]:
%timeit typed_array_op(x,y)

The slowest run took 8.50 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.82 µs per loop


In [ ]:
%%cython 
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def typed_unsafe_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result


In [ ]:
%timeit typed_unsafe_array_op(x,y)

The slowest run took 11.65 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 4.5 µs per loop


# Opdracht: np.vectorize naar Cython

In [ ]:
def complicated_calculation(x,y):
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*np.sin(y)
  else:
      res = cos(x/(0.1+abs(y)))
  return res

In [ ]:
def get_results_fast(x,y):
  return np.vectorize(complicated_calculation)(x,y)

In [ ]:
x = np.random.randn(int(1e6))
y = np.random.randn(int(1e6))

In [ ]:
%timeit res_fast = get_results_fast(x, y)

1 loop, best of 5: 610 ms per loop


In [ ]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport sin, cos, tan, fabs

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double complicated_calculation(double x, double y):
  cdef double res
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*sin(y)
  else:
      res = cos(x/(0.1+fabs(y)))
  return res

@cython.boundscheck(False)
@cython.wraparound(False)
def c_get_results_fast(double[:] x, double[:] y):
  cdef int n
  n = len(x)
  cdef double[:] res = np.zeros(n, dtype=np.dtype('d'))
  for i in range(n):
    res[i] = complicated_calculation(x[i],y[i])
  
  return res

In [ ]:
%timeit res_fast = c_get_results_fast(x, y)

10 loops, best of 5: 49.4 ms per loop


# Cython and Pandas

We downloaden eerst wat data...

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    195      0 --:--:--  0:00:01 --:--:--   195
100 3781k    0 3781k    0     0  1524k      0 --:--:--  0:00:02 --:--:-- 1524k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    218      0 --:--:--  0:00:01 --:--:--   218
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 3781k    0 3781k    0     0  1576k      0 --:--:--  0:00:02 --:--:-- 1576k
Archive:  weatherAUS.zip
  inflating: weatherAUS.csv          


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('weatherAUS.csv')

In [ ]:
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype == 'float64' else x,
              axis=0)
df.head()

In [ ]:
def average_value(values):
  return np.mean(values)


In [ ]:
%timeit df.head(100).apply(lambda x: average_value(x[11:13]), axis=1)


10 loops, best of 5: 20.6 ms per loop


In [ ]:
%%cython
import numpy as np

def c_average_value(values):
  return np.mean(values)


In [ ]:
%timeit df.head(100).apply(lambda x: c_average_value(x[11:13]), axis=1)

100 loops, best of 5: 17 ms per loop


In [ ]:
%%cython
import numpy as np
cimport numpy as np

def c_average_value_typed(int[:] values):
  return np.mean(values)

In [ ]:
%timeit df.head(100).apply(lambda x: c_average_value_typed(x[11:13].to_numpy(dtype=np.dtype('i'))), axis=1)

100 loops, best of 5: 6.23 ms per loop


TEKST OVER PROFILING

In [ ]:
%%cython
cimport cython
cimport numpy as np
import numpy as np

cdef int c_average_value_typed(int[:] values):
  return np.mean(values)

def c_apply_average_value(int[:,:] df_cols):
  n = df_cols.shape[0]
  cdef double[:] res = np.empty(n)
  for i in range(n):
    res[i] = c_average_value_typed(df_cols[i])

  return res

In [ ]:
%timeit c_apply_average_value(df.head(100)[df.columns[11:13]].to_numpy(dtype=np.dtype('i'))) 

1000 loops, best of 5: 1.48 ms per loop
